# Mulitple Pixel Qinj Mapping

## Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from hist import Hist
import datetime
import pandas as pd
from glob import glob
import mplhep as hep
hep.style.use("CMS")

### Board Name

In [ ]:
chip_name = "02D5#13"

### Read data

In [ ]:
# dir = '../../ETROC-Data/' + datetime.date.today().isoformat() + "_Array_Test_Results/"
dir = '../../inputs/2023-07-31_Array_Test_Results/'
files = glob(dir+f'*{chip_name}*/*translated*')
files

### Save into pandas dataframe

In [ ]:
d = []

for ifile in files:
    with open(ifile, "r") as infile:    
        for line in infile.readlines():
            if line.split(' ')[2] == 'HEADER':
                bcid = line.strip().split(' ')[-1]
            elif line.split(' ')[2] == 'DATA':
                col = int(line.split(' ')[6])
                row = int(line.split(' ')[8])
                toa = int(line.split(' ')[10])
                tot = int(line.split(' ')[12])
                cal = int(line.split(' ')[14])
                d.append(
                    {
                    'col': col,
                    'row': row,
                    'toa': toa,
                    'tot': tot,
                    'cal': cal,     
                    'bcid': bcid,    
                    }
                )
            elif line.split(' ')[2] == 'TRAILER':
                pass
            
df = pd.DataFrame(d)
df = df[df['cal']<220] # simple CAL code cut

### Print unique BCIDs

In [ ]:
value_counts = df['bcid'].value_counts()
for value, count in value_counts.items():
    print(f"{value}: {count}")

print(value_counts.keys())

### filter data with BCID cut

In [ ]:
# bcid_cuts = ['1854', '2087', '3522', '2699', '549', '527'] # 02D5_#12, 2023-07-26
# bcid_cuts = ['3558', '1472', '2964', '2425', '1414', '1676'] # 01E2_#52, 2023-07-27
# bcid_cuts = ['2209', '1237', '2278', '1180', '26', '1242'] # 01E2_#48, 2023-07-28
# bcid_cuts = ['2909', '1791', '2767', '1213', '218', '724'] # 02D5_#13, 2023-07-28

# 01E2_#48, 2023-07-31
# bcid_cuts = ['2722', '1378', '865', '1081', '2212', '2135', '3321', '361', '2462', '1776', '2744', '3103', '2086', '1641', '2709', '3069']

# 01E2_#52, 2023-07-31
# bcid_cuts = ['3484', '1619', '703', '3517', '90', '2881', '1964', '1645', '777', '12', '1806', '403', '2177', '2677', '109', '2954']

# 02D5_#12, 2023-07-31
# bcid_cuts = ['1927', '1933', '2378', '6', '1405', '719', '3347', '2760', '2194', '2488', '877', '2892', '121', '2852', '1617', '2673']

# 02D5_#13, 2023-07-31
bcid_cuts = ['3438', '1646', '2858', '2220', '2878', '3035', '3254', '1597', '2389', '664', '1576', '445', '3024', '1461', '1166', '2967']

filtered_data = df[df['bcid'].isin(bcid_cuts)]

unique_values = filtered_data['bcid'].unique()
print(len(unique_values))
print(unique_values)

### Delete big dataframe to save memory

In [ ]:
# Check the memory usage again
print(df.info())

# Check the memory usage again
filtered_data.drop(columns=['bcid'], inplace=True)
print(filtered_data.info())
  
# Delete the old DataFrame
del df

## Make 2D map of toa, tot, cal

In [ ]:
cals = filtered_data.groupby(["col", "row"]).agg({'cal':['mean','std']})
cals.columns = cals.columns.droplevel()
cals.reset_index(inplace=True)

table_mean = cals.pivot_table(index='row', columns='col', values='mean')
table_mean = table_mean.round(1)

table_std = cals.pivot_table(index='row', columns='col', values='std')
table_std = table_std.round(2)

In [ ]:
plt.rcParams["xtick.major.size"] = 2.5
plt.rcParams["ytick.major.size"] = 2.5
plt.rcParams['xtick.minor.visible'] = False
plt.rcParams['ytick.minor.visible'] = False


fig, axes = plt.subplots(1, 2, figsize=(20, 10))

im1 = axes[0].imshow(table_mean, vmin=1)
im2 = axes[1].imshow(table_std, vmin=1)

axes[0].set_title('CAL Mean')
axes[1].set_title('CAL Std')

axes[0].set_xticks(np.arange(0,16))
axes[0].set_yticks(np.arange(0,16))
axes[1].set_xticks(np.arange(0,16))
axes[1].set_yticks(np.arange(0,16))

axes[0].invert_xaxis()
axes[0].invert_yaxis()
axes[1].invert_xaxis()
axes[1].invert_yaxis()

cbar1 = fig.colorbar(im1, ax=axes[0], shrink=0.6) 
cbar2 = fig.colorbar(im2, ax=axes[1], shrink=0.6)

# i for col, j for row
for i in range(16):
    for j in range(16):
        if np.isnan(table_mean.iloc[i,j]): 
            continue
        axes[0].text(j, i, table_mean.iloc[i,j], ha="center", va="center", rotation=45, fontweight="bold", fontsize=10)

for i in range(16):
    for j in range(16):
        if np.isnan(table_std.iloc[i,j]): 
            continue
        axes[1].text(j, i, table_std.iloc[i,j], ha="center", va="center", rotation=45, color='white', fontweight="bold", fontsize=10)


plt.tight_layout()
fig.savefig(f'CAL_2D_Map_{chip_name}.png')

In [ ]:
toas = filtered_data.groupby(["col", "row"]).agg({'toa':['mean','std']})
toas.columns = toas.columns.droplevel()
toas.reset_index(inplace=True)

table_mean = toas.pivot_table(index='row', columns='col', values='mean')
table_mean = table_mean.round(1)

table_std = toas.pivot_table(index='row', columns='col', values='std')
table_std = table_std.round(2)

In [ ]:
plt.rcParams["xtick.major.size"] = 2.5
plt.rcParams["ytick.major.size"] = 2.5
plt.rcParams['xtick.minor.visible'] = False
plt.rcParams['ytick.minor.visible'] = False

fig, axes = plt.subplots(1, 2, figsize=(20, 10))

im1 = axes[0].imshow(table_mean, vmin=1)
im2 = axes[1].imshow(table_std, vmin=1)

axes[0].set_title('TOA Mean')
axes[1].set_title('TOA Std')

axes[0].set_xticks(np.arange(0,16))
axes[0].set_yticks(np.arange(0,16))
axes[1].set_xticks(np.arange(0,16))
axes[1].set_yticks(np.arange(0,16))

axes[0].invert_xaxis()
axes[0].invert_yaxis()
axes[1].invert_xaxis()
axes[1].invert_yaxis()

cbar1 = fig.colorbar(im1, ax=axes[0], shrink=0.6) 
cbar2 = fig.colorbar(im2, ax=axes[1], shrink=0.6)

# i for col, j for row
for i in range(16):
    for j in range(16):
        if np.isnan(table_mean.iloc[i,j]): 
            continue
        axes[0].text(j, i, table_mean.iloc[i,j], ha="center", va="center", rotation=45, fontweight="bold", fontsize=10)

for i in range(16):
    for j in range(16):
        if np.isnan(table_std.iloc[i,j]): 
            continue
        axes[1].text(j, i, table_std.iloc[i,j], ha="center", va="center", rotation=45, color='white', fontweight="bold", fontsize=10)

plt.tight_layout()
fig.savefig(f'TOA_2D_Map_{chip_name}.png')

In [ ]:
tots = filtered_data.groupby(["col", "row"]).agg({'tot':['mean','std']})
tots.columns = tots.columns.droplevel()
tots.reset_index(inplace=True)

table_mean = tots.pivot_table(index='row', columns='col', values='mean')
table_mean = table_mean.round(1)

table_std = tots.pivot_table(index='row', columns='col', values='std')
table_std = table_std.round(2)

In [ ]:
plt.rcParams["xtick.major.size"] = 2.5
plt.rcParams["ytick.major.size"] = 2.5
plt.rcParams['xtick.minor.visible'] = False
plt.rcParams['ytick.minor.visible'] = False


fig, axes = plt.subplots(1, 2, figsize=(20, 10))

im1 = axes[0].imshow(table_mean, vmin=1)
im2 = axes[1].imshow(table_std, vmin=1)

axes[0].set_title('TOT Mean')
axes[1].set_title('TOT Std')

axes[0].set_xticks(np.arange(0,16))
axes[0].set_yticks(np.arange(0,16))
axes[1].set_xticks(np.arange(0,16))
axes[1].set_yticks(np.arange(0,16))

axes[0].invert_xaxis()
axes[0].invert_yaxis()
axes[1].invert_xaxis()
axes[1].invert_yaxis()

cbar1 = fig.colorbar(im1, ax=axes[0], shrink=0.6) 
cbar2 = fig.colorbar(im2, ax=axes[1], shrink=0.6)

# i for col, j for row
for i in range(16):
    for j in range(16):
        if np.isnan(table_mean.iloc[i,j]): 
            continue
        axes[0].text(j, i, table_mean.iloc[i,j], ha="center", va="center", rotation=45, fontweight="bold", fontsize=10)

for i in range(16):
    for j in range(16):
        if np.isnan(table_std.iloc[i,j]): 
            continue
        axes[1].text(j, i, table_std.iloc[i,j], ha="center", va="center", rotation=45, color='white', fontweight="bold", fontsize=10)

plt.tight_layout()
fig.savefig(f'TOT_2D_Map_{chip_name}.png')

## 2D map with individual histogram

In [ ]:
hcal = [[(
        Hist.new.Integer(125, 185, name="CAL", label="CAL [LSB]")
        .Int64()
      ) if not np.isnan(table_mean[j][i]) else np.nan
      for j in range(16)] 
     for i in range(16)]

htoa = [[(
        Hist.new.Integer(185, 305, name="TOA", label="TOA [LSB]")
        .Int64()
      ) if not np.isnan(table_mean[j][i]) else np.nan
      for j in range(16)] 
     for i in range(16)]

htot = [[(
        Hist.new.Integer(45, 85, name="TOT", label="TOT [LSB]")
        .Int64()
      ) if not np.isnan(table_mean[j][i]) else np.nan
      for j in range(16)] 
     for i in range(16)]

In [ ]:
# i for col, j for row

for i in range(16):
    for j in range(16):
        cal_subset = filtered_data[(filtered_data['row'] == j) & (filtered_data['col'] == i)]['cal']
        toa_subset = filtered_data[(filtered_data['row'] == j) & (filtered_data['col'] == i)]['toa']
        tot_subset = filtered_data[(filtered_data['row'] == j) & (filtered_data['col'] == i)]['tot']
        if cal_subset.empty:
            continue
        elif np.isnan(hcal[j][i]).any():
            continue
        else:
            hcal[j][i].fill(cal_subset)
            htoa[j][i].fill(toa_subset)
            htot[j][i].fill(tot_subset)

In [ ]:
# Create subplots for each unique combination of 'row' and 'col'
nrows = 16
ncols = 16

# Create a figure and subplots
fig, axs = plt.subplots(nrows, ncols, dpi=150, figsize=(160, 160))

# i for col, j for row
for i in range(16):
    for j in range(16):
        if np.isnan(hcal[j][i]).any():
            axs[15-j][15-i].axis('off')
        else:
            hcal[j][i].project("CAL").plot1d(ax=axs[15-j][15-i], histtype="fill")
            hcal[j][i].project("CAL").plot1d(ax=axs[15-j][15-i], marker='.', histtype="errorbar", fillstyle='full', xerr=True, capsize=2, markersize=5)
            axs[15-j][15-i].set_title(f"Pixel ({j},{i}) CAL (LSB)", fontsize=22)
            axs[15-j][15-i].set_xticks(axs[15-j][15-i].get_xticks(), axs[15-j][15-i].get_xticklabels(), rotation='vertical')
            axs[15-j][15-i].minorticks_off()
            axs[15-j][15-i].set_xlim(np.argmax(hcal[j][i].values())-4, np.argmax(hcal[j][i].values())+4)

fig.savefig(f"CAL_individual_histogram_{chip_name}.pdf")
plt.close(fig)

In [ ]:
# Create subplots for each unique combination of 'row' and 'col'
nrows = 16
ncols = 16

# Create a figure and subplots
fig, axs = plt.subplots(nrows, ncols, dpi=150, figsize=(160, 160))

# i for col, j for row
for i in range(16):
    for j in range(16):
        if np.isnan(htoa[j][i]).any():
            axs[15-j][15-i].axis('off')
        else:
            htoa[j][i].project("TOA").plot1d(ax=axs[15-j][15-i], histtype="fill")
            htoa[j][i].project("TOA").plot1d(ax=axs[15-j][15-i], marker='.', histtype="errorbar", fillstyle='full', xerr=True, capsize=2, markersize=5)
            axs[15-j][15-i].set_title(f"Pixel ({j},{i}) TOA (LSB)", fontsize=22)
            axs[15-j][15-i].set_xticks(axs[15-j][15-i].get_xticks(), axs[15-j][15-i].get_xticklabels(), rotation='vertical')
            axs[15-j][15-i].minorticks_off()
            axs[15-j][15-i].set_xlim(np.argmax(htoa[j][i].values())-4, np.argmax(htoa[j][i].values())+4)

fig.savefig(f"TOA_individual_histogram_{chip_name}.pdf")
plt.close(fig)

In [ ]:
# Create subplots for each unique combination of 'row' and 'col'
nrows = 16
ncols = 16

# Create a figure and subplots
fig, axs = plt.subplots(nrows, ncols, dpi=150, figsize=(160, 160))

# i for col, j for row
for i in range(16):
    for j in range(16):
        if np.isnan(htot[j][i]).any():
            axs[15-j][15-i].axis('off')
        else:
            htot[j][i].project("TOT").plot1d(ax=axs[15-j][15-i], histtype="fill")
            htot[j][i].project("TOT").plot1d(ax=axs[15-j][15-i], marker='.', histtype="errorbar", fillstyle='full', xerr=True, capsize=2, markersize=5)
            axs[15-j][15-i].set_title(f"Pixel ({j},{i}) TOT (LSB)", fontsize=22)
            axs[15-j][15-i].set_xticks(axs[15-j][15-i].get_xticks(), axs[15-j][15-i].get_xticklabels(), rotation='vertical')
            axs[15-j][15-i].minorticks_off()
            axs[15-j][15-i].set_xlim(np.argmax(htot[j][i].values())-4, np.argmax(htot[j][i].values())+4)

fig.savefig(f"TOT_individual_histogram_{chip_name}.pdf")
plt.close(fig)